In [15]:
import torch 
import torch.nn as nn 
import transformers 
from collections import OrderedDict

In [3]:
bert = transformers.BertModel.from_pretrained('hfl/chinese-macbert-base', cache_dir='data/pretrained_model/macbert_base')

Some weights of the model checkpoint at hfl/chinese-macbert-base were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [17]:
state_dict = bert.state_dict()
names = list(state_dict.keys())
new_dict = OrderedDict()
for name in names:
    if name.startswith('encoder.'):
        new_dict['fusebert.'+name] = state_dict[name]

In [21]:
model.load_state_dict(new_dict, strict=False)

_IncompatibleKeys(missing_keys=['cls_token', 'splitbert.embeddings.word_embeddings.weight', 'splitbert.embeddings.LayerNorm.weight', 'splitbert.embeddings.LayerNorm.bias', 'fusebert.embeddings.token_type_embeddings.weight', 'fusebert.embeddings.LayerNorm.weight', 'fusebert.embeddings.LayerNorm.bias', 'image_encoder.0.weight', 'image_encoder.0.bias', 'image_encoder.1.weight', 'image_encoder.1.bias', 'head.weight', 'head.bias'], unexpected_keys=[])

In [5]:
import os
import itertools
import torch 
import json
import numpy as np 
import random 
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm 
import copy

from model.bert.bertconfig import BertConfig
from model.fusemodel import FuseModel

split_layers = 0
fuse_layers = 12
n_img_expand = 6

vocab_file = 'dataset/vocab/vocab.txt'

split_config = BertConfig(num_hidden_layers=split_layers)
fuse_config = BertConfig(num_hidden_layers=fuse_layers)
model = FuseModel(split_config, fuse_config, vocab_file, n_img_expand=n_img_expand)